# In this Notebook:- 
* We fine-tuned the IndicTrans2-en-indic-dist-200M model on the PHINC (Parallel Hinglish Social Media Code-Mixed Corpus for Machine Translation) for the EN→HINGLISH translation task. The resulting model was evaluated using a comprehensive suite of MT quality metrics, including BLEU, ChrF, COMET, BERTScore, and BLEURT.

[](http://)

In [15]:
!pip uninstall -y numpy scipy pandas pyarrow datasets transformers
!pip install --force-reinstall --no-cache-dir \
  numpy==1.26.4 \
  scipy==1.11.4 \
  pandas==2.1.4 \
  pyarrow==14.0.2 \
  datasets==2.16.1 \
  transformers==4.36.2


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.11.4
Uninstalling scipy-1.11.4:
  Successfully uninstalled scipy-1.11.4
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: datasets 2.16.1
Uninstalling datasets-2.16.1:
  Successfully uninstalled datasets-2.16.1
Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 285.5 MB/s

In [16]:
#Checking wheather GPU is working or not
!nvidia-smi


Thu Dec 18 13:35:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
# installing dataset and transformer
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [18]:
# to remove version conflict of Protobuf so, downgrade version of Protobuf
!pip install protobuf==3.20.3 

In [19]:
# Importing all required modules
import os
import sys
import transformers
import torch  # pytorch Import
import sacrebleu
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_dataset # for loading the dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # For getting Embedding
from transformers import DataCollatorForSeq2Seq #getting sequential model and collator for loading batchwise of data
from torch.optim import AdamW # Optimizer



# Indictrans2-en-indic-dist-200M Model
* source: https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M

In [20]:
# Enter Access Token and rerun
from huggingface_hub import login
login(new_session=False)

# Note:
* I was using the free version of Kaggle, and the memory limit was getting exhausted while training the 1B-parameter model. Because of this constraint, I switched to using the 200M-parameter model instead.

In [21]:
ckpt = "ai4bharat/indictrans2-en-indic-dist-200M" # Model Checkpoint 

model = AutoModelForSeq2SeqLM.from_pretrained(
    ckpt,
    trust_remote_code=True,                                         
)

tokenizer = AutoTokenizer.from_pretrained(
    ckpt,
    trust_remote_code=True
)

# Move safely to GPU
model = model.to(torch.float16).to("cuda")   

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

...

dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

model.SRC:   0%|          | 0.00/759k [00:00<?, ?B/s]

model.TGT:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

# The Dataset¶

* Source: https://huggingface.co/datasets/LingoIITGN/PHINC

In [22]:
raw_dataset = load_dataset("LingoIITGN/PHINC")

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'English_Translation'],
        num_rows: 13738
    })
})

In [24]:
def convert_to_translation(example):
    return {
        "translation": {
            "en": example["English_Translation"],
            "hing": example["Sentence"]
        }
    }

raw_dataset = raw_dataset.map(convert_to_translation)

Map:   0%|          | 0/13738 [00:00<?, ? examples/s]

In [25]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'English_Translation', 'translation'],
        num_rows: 13738
    })
})

In [26]:
from datasets import DatasetDict

# total rows
total_rows = raw_dataset["train"].num_rows

# required splits
train_rows = 12000
valid_rows = 538
test_rows = 1200

# slice the dataset
train_dataset = raw_dataset["train"].select(range(0, train_rows))
valid_dataset = raw_dataset["train"].select(range(train_rows, train_rows + valid_rows))
test_dataset  = raw_dataset["train"].select(range(train_rows + valid_rows,
                                                  train_rows + valid_rows + test_rows))

# create final DatasetDict
final_dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

final_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'English_Translation', 'translation'],
        num_rows: 12000
    })
    validation: Dataset({
        features: ['Sentence', 'English_Translation', 'translation'],
        num_rows: 538
    })
    test: Dataset({
        features: ['Sentence', 'English_Translation', 'translation'],
        num_rows: 1200
    })
})

In [27]:
raw_dataset = final_dataset

# Observation for Statistics related to dataset

In [28]:
# Import required libraries
import numpy as np
import math
import nltk
nltk.download("punkt")  # one-time

def add_stats(example):
    text = example["translation"]["en"]
    # guard
    if text is None: text = ""
    text = text.strip() # Removes unwanted spacing
    words = text.split()
    # sentence count (approx)
    sents = nltk.tokenize.sent_tokenize(text) if text else []
    example["num_words"] = len(words)
    example["num_chars"] = len(text)
    example["num_sentences"] = len(sents)
    return example

raw_dataset = raw_dataset.map(add_stats, batched=False)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [29]:
# Obtaining the Statistics:

def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))


=== TRAIN ===
Words: {'count': 12000, 'min': 1, 'p1': 1, 'p10': 5, 'median': 11.0, 'mean': 12.335166666666666, 'std': 6.736442926764506, 'p90': 22, 'p99': 31, 'max': 46}
Chars: {'count': 12000, 'min': 1, 'p1': 1, 'p10': 30, 'median': 67.0, 'mean': 74.00516666666667, 'std': 38.514235116887484, 'p90': 130, 'p99': 167, 'max': 278}
Sentences: {'count': 12000, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.5585, 'std': 0.989062392706682, 'p90': 3, 'p99': 5, 'max': 16}

=== VALIDATION ===
Words: {'count': 538, 'min': 1, 'p1': 1, 'p10': 5, 'median': 11.0, 'mean': 11.962825278810408, 'std': 6.470587802969751, 'p90': 21, 'p99': 28, 'max': 36}
Chars: {'count': 538, 'min': 1, 'p1': 1, 'p10': 28, 'median': 69.0, 'mean': 73.38289962825279, 'std': 39.0196100416231, 'p90': 125, 'p99': 169, 'max': 221}
Sentences: {'count': 538, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.3717472118959109, 'std': 0.694709906589641, 'p90': 2, 'p99': 4, 'max': 6}

=== TEST ===
Words: {'count': 1200, 'mi

In [30]:
from datasets import DatasetDict

In [31]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["en"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

Filter:   0%|          | 0/12000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/538 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [32]:
type(raw_dataset)

datasets.dataset_dict.DatasetDict

In [33]:
# compute p99 threshold('p99': 31 so, removing other outliers(longer than 31 words))
word_lengths = np.array(raw_dataset["train"]["num_words"])
p99_threshold = int(np.percentile(word_lengths, 99))
print("Removing sentences longer than:", p99_threshold, "words")
raw_dataset["train"] = raw_dataset["train"].filter(
    lambda ex: ex["num_words"] <= p99_threshold
)


Removing sentences longer than: 31 words


Filter:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [34]:
# Obtaining the desired Statistics
def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))



=== TRAIN ===
Words: {'count': 11912, 'min': 1, 'p1': 1, 'p10': 5, 'median': 11.0, 'mean': 12.170752182672935, 'std': 6.478840709255029, 'p90': 22, 'p99': 29, 'max': 31}
Chars: {'count': 11912, 'min': 1, 'p1': 1, 'p10': 30, 'median': 67.0, 'mean': 73.26544660846206, 'std': 37.62196715365125, 'p90': 129, 'p99': 161, 'max': 278}
Sentences: {'count': 11912, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.542981867024849, 'std': 0.9439458074386832, 'p90': 3, 'p99': 5, 'max': 12}

=== VALIDATION ===
Words: {'count': 538, 'min': 1, 'p1': 1, 'p10': 5, 'median': 11.0, 'mean': 11.962825278810408, 'std': 6.470587802969751, 'p90': 21, 'p99': 28, 'max': 36}
Chars: {'count': 538, 'min': 1, 'p1': 1, 'p10': 28, 'median': 69.0, 'mean': 73.38289962825279, 'std': 39.0196100416231, 'p90': 125, 'p99': 169, 'max': 221}
Sentences: {'count': 538, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.3717472118959109, 'std': 0.694709906589641, 'p90': 2, 'p99': 4, 'max': 6}

=== TEST ===
Words: {'count'

In [35]:
# Sample Example
raw_dataset['train'][0]

{'Sentence': "@someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how",
 'English_Translation': "@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show",
 'translation': {'en': "@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show",
  'hing': "@someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how"},
 'num_words': 19,
 'num_chars': 126,
 'num_sentences': 1}

In [36]:
from datasets import DatasetDict

# New desired sizes
N_TRAIN = 2000
N_VAL   = 150
N_TEST  = 250

# Downsample using .select()
small_train = raw_dataset["train"].select(range(N_TRAIN))
small_val   = raw_dataset["validation"].select(range(N_VAL))
small_test  = raw_dataset["test"].select(range(N_TEST))

# Create a new DatasetDict
small_dataset = DatasetDict({
    "train": small_train,
    "validation": small_val,
    "test": small_test
})

small_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'English_Translation', 'translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['Sentence', 'English_Translation', 'translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 150
    })
    test: Dataset({
        features: ['Sentence', 'English_Translation', 'translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 250
    })
})

In [37]:
model.eval() # Evaluation of model


IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(32322, 512, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_

In [38]:
# Hinglish --> Hindi Translation

SRC_TAG = "eng_Latn"   # Hinglish / Roman Hindi
TGT_TAG = "hin_Deva"  # Hindi
def hinglish_to_hindi(sentences, batch_size=8, max_len=128):
    outputs = []

    device = model.device

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        tagged = [
            f"{SRC_TAG} {TGT_TAG} {text}"
            for text in batch
        ]

        inputs = tokenizer(
            tagged,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len
        ).to(device)

        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_length=max_len,
                num_beams=4,
                use_cache=False,       
                early_stopping=True
            )

        decoded = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True
        )

        outputs.extend(decoded)

    return outputs


In [39]:
def convert_split(split):
    hinglish_sentences = [ex["hing"] for ex in split["translation"]]
    hindi_sentences = hinglish_to_hindi(hinglish_sentences)

    new_translation = []
    for i in range(len(split)):
        new_translation.append({
            "en": split["translation"][i]["en"],
            "hing": split["translation"][i]["hing"],
            "hi": hindi_sentences[i]
        })

    return split.remove_columns("translation").add_column(
        "translation", new_translation
    )


In [40]:
small_dataset["train"] = convert_split(small_dataset["train"])
small_dataset["validation"] = convert_split(small_dataset["validation"])
small_dataset["test"] = convert_split(small_dataset["test"])


Flattening the indices:   0%|          | 0/2000 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/150 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/250 [00:00<?, ? examples/s]

In [41]:
small_dataset["train"]["translation"][0]

{'en': "@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show",
 'hi': '@someUSER आपको ब्रिटिश बाल गायक सोफिया ग्रेस और रोजी की अपने शो की यात्रा की पहली वर्षगांठ मनाने के लिए बधाई ।',
 'hing': "@someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how"}

In [42]:
small_dataset

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'English_Translation', 'num_words', 'num_chars', 'num_sentences', 'translation'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['Sentence', 'English_Translation', 'num_words', 'num_chars', 'num_sentences', 'translation'],
        num_rows: 150
    })
    test: Dataset({
        features: ['Sentence', 'English_Translation', 'num_words', 'num_chars', 'num_sentences', 'translation'],
        num_rows: 250
    })
})

In [43]:
raw_dataset = small_dataset  # As said in Task 2(for 2000 pair of sentence)

# Applying Tokenization:- How we obtain Embedding
* Attention Mask = Padding Mask x look Ahead Mask
* Input_ids = input_ids are tokenized text converted into numeric indices from tokenizer vocabulary.
* The model converts input_ids to embeddings internally through an embedding layer.

# Pipeline
* Text → Tokens → IDs → Embeddings → Transformer
* "I love India"
*      ↓              (tokenization)
* ["I","love","India"]
*      ↓              (vocab lookup)
* [34, 91, 2563]  ← input_ids
*      ↓
* [embedding vectors] ← actual embeddings used by model

In [46]:
# sample Example;
text = "Hello Myself Virendra. A final year student at NIT Surat."
tokenizer("eng_Latn hin_Deva " + text)

{'input_ids': [4, 15, 7951, 23463, 8660, 11258, 5933, 85, 55, 910, 195, 1410, 48, 349, 6601, 8308, 85, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [47]:
# Tokenize the Target language(Hinglish) - Example
with tokenizer.as_target_tokenizer():
    print(tokenizer("Gud mrng sir aapko Mahashivratri ki hardik mangalkamnaye"))

{'input_ids': [2513, 36544, 5506, 7881, 49984, 79063, 4586, 25487, 54737, 79673, 61438, 8014, 51561, 14775, 60875, 59614, 32513, 85120, 6473, 81543, 7130, 15937, 5172, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


 # Preprocess Fxn for Tokenization:

In [48]:
# tags for IndicTrans2 English->Hindi
SRC_TAG = "eng_Latn"
TGT_TAG = "hin_Deva"

max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):

    inputs = [f"{SRC_TAG} {TGT_TAG} {ex[source_lang].strip() if ex[source_lang] else ''}"
              for ex in examples["translation"]]
    targets = [ex[target_lang].strip() if ex[target_lang] else "" 
               for ex in examples["translation"]]

    # tokenize source (each string already prefixed with tags)
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=True,   
    )

    # Tokenizer for Target lang(Hinglish)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [f"{TGT_TAG} {t}" if t else f"{TGT_TAG} " for t in targets],
            max_length=max_target_length,
            truncation=True,
            padding=True,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [49]:
# Obtaining token of two rows of Train dataset
print(preprocess_function(raw_dataset["train"][:2])) 


{'input_ids': [[1, 1, 1, 1, 4, 15, 5912, 23, 3468, 964, 8063, 31, 22, 6070, 1197, 4363, 14055, 21086, 18902, 4638, 23, 10, 10190, 1567, 4638, 23, 204, 811, 3096, 743, 8, 63, 654, 2], [4, 15, 5912, 27756, 20257, 25260, 15664, 71, 114, 14, 201, 90, 22, 26, 3, 49, 36, 5705, 305, 82, 469, 42, 514, 2962, 17935, 5051, 328, 3204, 947, 876, 1075, 16082, 832, 2]], 'attention_mask': [[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[105948, 59836, 2134, 5172, 43144, 14066, 1869, 48770, 5658, 24159, 3532, 708, 4657, 1772, 9478, 26131, 438, 350, 10241, 17, 8323, 12, 219, 1735, 12, 2153, 12, 1972, 559, 33391, 25137, 9, 44, 5541, 6, 2, 1, 1, 1], [105948, 59836, 2134, 5172, 43144, 14066, 273, 9937, 14066, 97, 6355, 1084, 6355, 888, 44, 51, 8864, 2220, 252, 321, 4286, 170, 100, 42507, 458, 20, 163, 36, 26535, 4443, 13326, 409, 20866,

In [50]:
# Setting  hyperparameter Values
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [51]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [52]:
# 1) Tokenization of dataset and DataCollator
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True,
                                    remove_columns=raw_dataset["train"].column_names)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

# 2)train_dataloader and test_dataloader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size,
                              shuffle=True, collate_fn=data_collator, num_workers=4)
validation_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size,
                                   shuffle=False, collate_fn=data_collator, num_workers=2)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [53]:
model.float()        # make sure parameters are float32
model.to(device)

# 3) Recreate optimizer (must be created after model param dtypes are finalized)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 4) Mixed precision utilities
scaler = GradScaler()
grad_accum = 2
num_epochs = max(1, int(num_train_epochs))

allowed_keys = {"input_ids", "attention_mask", "labels", "decoder_input_ids", "decoder_attention_mask"}

In [54]:
# 5) Training loop
model.train()
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    steps = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        # keep only model inputs and move to device
        model_batch = {k: v.to(device) for k, v in batch.items() if k in allowed_keys and isinstance(v, torch.Tensor)}

        with autocast("cuda"):                  # activations in fp16, params stay fp32
            outputs = model(**model_batch)
            loss = outputs.loss / grad_accum

        scaler.scale(loss).backward()

        if (steps + 1) % grad_accum == 0:
            # Unscale the Optimizer
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item() * grad_accum
        steps += 1

    avg_train = running_loss / max(1, steps)
    print(f"Train loss: {avg_train:.4f}")

    # 6) Validation
    model.eval()
    vloss = 0.0
    vsteps = 0
    with torch.no_grad():
        for vb in tqdm(validation_dataloader, desc="Validation"):
            vb_batch = {k: v.to(device) for k, v in vb.items() if k in allowed_keys and isinstance(v, torch.Tensor)}
            with autocast("cuda"):
                out = model(**vb_batch)
            vloss += out.loss.item()
            vsteps += 1
    vloss = vloss / max(1, vsteps)
    print(f"Validation loss: {vloss:.4f}")
    model.train()

print("Training finished.")



Epoch 1/1


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Train loss: 9.1854


Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Validation loss: 7.9155
Training finished.


In [248]:
URL_PATTERN = r"https?://\S+"
HANDLE_PATTERN = r"@\w+"
TECH_WORDS = [
    "ai/ml",
    "ai",
    "ml",
    "artificial intelligence",
    "machine learning",
    "data science",
    "deep learning"
]
SOCIAL_WORDS = [
    "really",
    "amazing",
    "awesome",
    "emotional",
    "touching",
    "bhai",
    "and",
    "sir",
    "madam",
    "fan",
    "fans",
    "love",
    "respect",
    "support"
]

WORD_PATTERN = r"\b(" + "|".join(map(re.escape, TECH_WORDS + SOCIAL_WORDS)) + r")\b"


In [249]:
import re
import torch

# ----------------------------------
# Token protection (ROBUST)
# ----------------------------------
def protect_tokens(text):
    protected = {}
    idx = 0
    patterns = [
    URL_PATTERN,       # URLs
    HANDLE_PATTERN,    # @handles
    WORD_PATTERN       # single words (tech + social)
   ]
   
    for pattern in patterns:
        def repl(match):
            nonlocal idx
            key = f"XQZPLCH{idx}XQZ"
            protected[key] = match.group()
            idx += 1
            return key

        text = re.sub(pattern, repl, text, flags=re.IGNORECASE)

    return text, protected


def restore_tokens(text, protected):
    for k, v in protected.items():
        text = text.replace(k, v)
    return text


In [274]:
def en_to_hi(
    text,
    model,
    tokenizer,
    src_tag="eng_Latn",
    tgt_tag="hin_Deva",
    max_length=128
):
    # Protect non-linguistic / technical tokens
    safe_text, protected = protect_tokens(text)

    #  Add language tags
    tagged = f"{src_tag} {tgt_tag} {safe_text}"

    #  Tokenize
    inputs = tokenizer(tagged, return_tensors="pt").to(model.device)

    #  Generate (IndicTrans2-safe settings)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,       # REQUIRED
            use_cache=False   # REQUIRED
        )

    #  Decode Hindi
    hindi_text = tokenizer.decode(out[0], skip_special_tokens=True)

    # Restore protected tokens
    hindi_text = restore_tokens(hindi_text, protected)

    return hindi_text

In [275]:
text= "@BeingSalmanKhan brother you are making us emotional. because of  few people you are just leaving all of us."
Hindi_text = en_to_hi(text, model, tokenizer)  
Hindi_text

'@BeingSalmanKhan भाई आप हमें emotional बना रहे हैं । कुछ लोगों की वजह से आप हम सभी को छोड़ रहे हैं ।'

In [276]:
import re

TECH_WORDS = [
    "ai", "ml", "ai/ml",
    "artificial intelligence",
    "machine learning",
    "emotional"
]

def protect_technical_words(text):
    protected = {}
    for i, word in enumerate(TECH_WORDS):
        token = f"<TECH{i}>"
        pattern = re.compile(re.escape(word), re.IGNORECASE)
        if pattern.search(text):
            protected[token] = word
            text = pattern.sub(token, text)
    return text, protected

def restore_technical_words(text, protected):
    for token, word in protected.items():
        text = text.replace(token.lower(), word)
        text = text.replace(token, word) 
    return text


In [277]:
def iast_to_hinglish(text):
    # --- diacritics ---
    diacritics = {
        "ā": "a",
        "ī": "i",
        "ū": "u",
        "ṃ": "n",
        "ṅ": "ng",
        "ñ": "ny",
        "ṭ": "t",
        "ḍ": "d",
        "ṇ": "n",
        "ś": "sh",
        "ṣ": "sh",
        "ṛ": "r",
        "ḥ": "h",
    }
    for k, v in diacritics.items():
        text = text.replace(k, v)

    # --- Hinglish lexical fixes ---
    fixes = {
        " nama ": " naam ",
        " naama ": " naam ",
        " vartamana ": " vartaman ",
        " vartamaana ": " vartaman ",
        " men ": " mein ",
        " hu~": " hoon",
        " hun": " hoon",
        " aura ": " aur ",
        " yaha ": " yeh ",
        " bahuta ": " bahut ",
        " anubhava ": " anubhav ",
        "|": ".",
    }
    for k, v in fixes.items():
        text = text.replace(k, v)

    return text.lower().strip()


In [278]:
!pip install indic-transliteration

In [279]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def hindi_to_hinglish(hindi_text):
    # protect technical terms
    safe_text, protected = protect_technical_words(hindi_text)

    # Hindi → IAST
    iast = transliterate(safe_text, sanscript.DEVANAGARI, sanscript.IAST)

    # IAST → Hinglish
    hinglish = iast_to_hinglish(iast)

    # restore technical terms
    hinglish = restore_technical_words(hinglish, protected)

    return hinglish


In [280]:
print(hindi_to_hinglish(Hindi_text))

@beingsalmankhan bhai apa hamen emotional bana rahe hain . kucha logon ki vajaha se apa hama sabhi ko chor̤a rahe hain .


# Evaluation metrics

# Finding  BLEU AND CHRF:

1. BLEU: BLEU checks how many n-grams from the candidate sentence also appear in the reference sentence.
2. CHRF: Instead of words, CHRF compares character n-grams.

In [281]:
model.device

device(type='cuda', index=0)

In [282]:
len(raw_dataset["test"])

250

In [ ]:
n_samples = 10

# determine model device safely
try:
    model_device = next(model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Model device:", model_device)

preds = []
refs = []

for i in range(min(n_samples, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # handle multiple possible row formats
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        # trans might be a dict or a string; handle both
        if isinstance(trans, dict):
            eng = trans.get("en") or trans.get("eng") or ""
            hi_ref = trans.get("hing") or trans.get("hin") or ""
        else:
            # sometimes translation is a string (rare) — treat as source
            eng = str(trans)
            hi_ref = ""
    elif isinstance(row, dict):
        # maybe keys are directly 'en' and 'hi'
        eng = row.get("en") or row.get("eng") or row.get("source") or ""
        hi_ref = row.get("hing") or row.get("hin") or row.get("target") or ""
    else:
        # fallback: row itself might be the translation dict-like
        try:
            eng = row["en"]
            hi_ref = row["hing"]
        except Exception:
            # last resort: stringify
            eng = str(row)
            hi_ref = ""

    eng = (eng or "").strip()
    hi_ref = (hi_ref or "").strip()
    refs.append(hi_ref if hi_ref else "")  # keep alignment
    safe_text, protected = protect_tokens(eng)
    # add required tags
    tagged = f"{SRC_TAG} {TGT_TAG} {eng}"

    # tokenize -> torch tensors -> move to model device
    tokenized = tokenizer(tagged, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = model.generate(
           **tokenized,
            max_length=128,
            num_beams=4,        
            use_cache=False,  
            early_stopping=True
        )
    
    Hindi_text = en_to_hi(eng, model, tokenizer)    

    hinglish = hindi_to_hinglish(Hindi_text)
    preds.append(hinglish)




Model device: cuda:0


In [284]:
preds[0:4]

['@beingsalmankhan bhai apa hamen emotional bana rahe hain . kucha logon ke karana apa hama sabhi ko chor̤a rahe hain .',
 'really touching and amazing. mujhe bacapana ki yada dilaya .',
 '@0__1 unake hatha and unake samana jo bhi ve karate hain vaha una para nirbhara karata hai .',
 '@trisha_naik elpha ne mahana rshiyon ka dhyana bhatakaya hai .']

In [285]:
len(preds)


10

In [287]:
#BLEU:
bleu = sacrebleu.corpus_bleu(preds, [refs])
#CHRF:
chrf = sacrebleu.corpus_chrf(preds, [refs])

print(f"\nEvaluated {len(preds)} samples")
print("BLEU:", bleu.score)
print("CHRF:", chrf.score)

for i in range(min(5, len(preds))):
    print(f"\n=== SAMPLE {i+1} ===")
    print("SRC :", (raw_dataset["test"][i].get("translation", raw_dataset["test"][i]).get("en")
                    if isinstance(raw_dataset["test"][i], dict) and "translation" in raw_dataset["test"][i]
                    else (raw_dataset["test"][i].get("en") if isinstance(raw_dataset["test"][i], dict) else str(raw_dataset["test"][i]))))
    print("PRED:", preds[i])
    print("REF :", refs[i])



Evaluated 10 samples
BLEU: 11.522239129499384
CHRF: 33.45488174566334

=== SAMPLE 1 ===
SRC : @BeingSalmanKhan brother you are making us emotional. because of  few people you are just leaving all of us.
PRED: @beingsalmankhan bhai apa hamen emotional bana rahe hain . kucha logon ke karana apa hama sabhi ko chor̤a rahe hain .
REF : @BeingSalmanKhan bhai ab aap emotional kar rahe ho. Kuch logo ki wajah se sabko chhod ke chale jaoge !

=== SAMPLE 2 ===
SRC : really touching and amazing. reminded me of my childhood.
PRED: really touching and amazing. mujhe bacapana ki yada dilaya .
REF : really touching and amazing .  .  bachpan ki yaad aa gayi .  .  .

=== SAMPLE 3 ===
SRC : @0__1 their hand and their luggage, it is upto them whatever they do.
PRED: @0__1 unake hatha and unake samana jo bhi ve karate hain vaha una para nirbhara karata hai .
REF : @0__1 unka haath, aur unka samaan jo chahe kare

=== SAMPLE 4 ===
SRC : @trisha_naik  Elf  has distracted the great sages.
PRED: @trisha_naik e

# Finding BERTScore:

* BERTScore: Uses BERT (or RoBERTa, or mBERT) embeddings to compare every token in candidate with every token in reference.

In [288]:
!pip install bert-score
from bert_score import score

P, R, F1 = score(preds, refs, lang="eng")
print("BERTScore F1:", F1.mean().item())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BERTScore F1: 0.7211230993270874


# Finding BLEURT:
* BLEURT computes a similarity score using a fine-tuned BERT model that predicts human judgment of translation quality.

In [263]:
# Required Packages for Bleurt
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-1b80ub54
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-1b80ub54
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [141]:
# Calculating Bleurt
import evaluate
bleurt = evaluate.load("bleurt")
results = bleurt.compute(predictions=preds, references=refs)
print(results)

Using default checkpoint 'bleurt-base-128' for sequence maximum length 128. You can use a bigger model for better results with e.g.: evaluate.load('bleurt', config_name='bleurt-large-512').


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


I0000 00:00:1766068126.935431     222 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2346 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766068126.939369     222 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


{'scores': [-0.3804433047771454, -0.5875755548477173, 0.025112386792898178, -0.1274239718914032, -0.41122981905937195, -0.13586753606796265, -1.0298868417739868, -0.8302490711212158, -1.600035548210144, -0.4865165948867798, -0.43631744384765625, -0.0038480497896671295, -0.6158275008201599, -0.7650679349899292, -0.4761647880077362, -0.906033456325531, -0.5395939946174622, -0.5018869042396545, -1.2332695722579956, -0.7155565023422241, -0.641909658908844, -1.3033435344696045, -0.48062774538993835, -1.6419179439544678, -0.8180795311927795, -1.4687597751617432, -0.935701847076416, -0.7306153178215027, -0.4692401587963104, -0.060972604900598526, -0.7518864870071411, -0.7861565947532654, -0.06227077916264534, -0.45538830757141113, -1.8166522979736328, -0.45611000061035156, -0.527264416217804, -0.5909863114356995, -1.8587026596069336, -1.099261999130249, -0.6316080689430237, -0.8847617506980896, -0.017818201333284378, -0.31022441387176514, -1.09712815284729, -0.3065151572227478, -1.44213819503

# Finding COMET
* COMET predicts a score that strongly correlates with human judgment.

In [289]:
# Required package for Comet
!pip install -q unbabel-comet
from comet import download_model, load_from_checkpoint


# choose model variable 
translation_model = globals().get("translation_model", None) or globals().get("model", None)
if translation_model is None:
    raise ValueError("No translation model found. Load your model into `model` or `translation_model` first.")

# device: try to get model device (handles DeviceMap too)
try:
    model_device = next(translation_model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_TAG = "eng_Latn"
TGT_TAG = "hin_Deva"

srcs = []
preds = []
refs = []

n = 10
for i in range(min(n, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # robust extraction of source & reference
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        if isinstance(trans, dict):
            src = (trans.get("en") or trans.get("eng") or "").strip()
            ref = (trans.get("hing") or trans.get("hing") or "").strip()
        else:
            src = str(trans).strip()
            ref = ""
    elif isinstance(row, dict):
        src = (row.get("en") or row.get("eng") or row.get("source") or "").strip()
        ref = (row.get("hing") or row.get("hing") or row.get("target") or "").strip()
    else:
        # fallback
        src = str(row).strip()
        ref = ""

    srcs.append(src)
    refs.append(ref)

    # add language tags required by IndicTrans2
    tagged = f"{SRC_TAG} {TGT_TAG} {src}"

    # tokenize -> PyTorch tensors -> move to model device
    tokenized = tokenizer(tagged,
                          return_tensors="pt",
                          truncation=True,
                          padding=True,
                          max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = translation_model.generate(**tokenized, max_length=128, num_beams=4, early_stopping=True)

    Hindi_text = en_to_hi(eng, model, tokenizer)    

    hinglish = hindi_to_hinglish(Hindi_text)
    preds.append(hinglish)

# ---- COMET evaluation ----
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# prepare data list for COMET
data = [{"src": s, "mt": p, "ref": r} for s, p, r in zip(srcs, preds, refs)]
# note: comet_model.predict returns a dict; 'scores' contains numeric values
comet_out = comet_model.predict(data, batch_size=8)
comet_scores = comet_out["scores"] if isinstance(comet_out, dict) and "scores" in comet_out else comet_out

print("Samples evaluated:", len(preds))
print("Mean COMET score:", float(sum(comet_scores) / len(comet_scores)))

# preview
for i in range(5):
    print(f"\n--- SAMPLE {i+1} ---")
    print("SRC :", srcs[i])
    print("PRED:", preds[i])
    print("REF :", refs[i])


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytor

Samples evaluated: 10
Mean COMET score: 0.45171141922473906

--- SAMPLE 1 ---
SRC : @BeingSalmanKhan brother you are making us emotional. because of  few people you are just leaving all of us.
PRED: ghrnita apa isa taraha se bata karenge to apake satha kauna baithega and biyara pi lo
REF : @BeingSalmanKhan bhai ab aap emotional kar rahe ho. Kuch logo ki wajah se sabko chhod ke chale jaoge !

--- SAMPLE 2 ---
SRC : really touching and amazing. reminded me of my childhood.
PRED: ghrnita apa isa taraha se bata karenge to kauna apake satha baithega and biyara pi lo
REF : really touching and amazing .  .  bachpan ki yaad aa gayi .  .  .

--- SAMPLE 3 ---
SRC : @0__1 their hand and their luggage, it is upto them whatever they do.
PRED: ghrnapurna. apa isa taraha se bata karenge. to kauna apake satha baithega and biyara pi lo.
REF : @0__1 unka haath, aur unka samaan jo chahe kare

--- SAMPLE 4 ---
SRC : @trisha_naik  Elf  has distracted the great sages.
PRED: ghrnita apa isa taraha se bata ka

In [290]:
# Saving the Model and tokenizer
model.save_pretrained("pt_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/dict.SRC.json',
 'tokenizer/dict.TGT.json',
 'tokenizer/model.SRC',
 'tokenizer/model.TGT',
 'tokenizer/added_tokens.json')